In [1]:

import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [2]:
import json

with open('/content/queries (2).json') as file:
    data = json.load(file)

training_sentences = []
training_labels = []
labels = []
responses = []
image_urls = []  # List to store image URLs

for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])

    responses.append(intent['responses'])

    # Check if "image_url" exists in the intent
    image_url = intent.get('image_url', None)
    image_urls.append(image_url)

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

num_classes = len(labels)

# Now you have training data, labels, responses, and image_urls


In [3]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [4]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [5]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

model.summary()
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 35)                595       
                                                                 
Total params: 17139 (66.95 KB)
Trainable params: 17139 (66.95 KB)
Non-trainable params: 0 (0.00 Byte)
____________________

In [9]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('/content/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# to save the fitted label encoder
with open('/content/label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
!pip install colorama

In [8]:
l

NameError: name 'l' is not defined

In [12]:
import spacy
import json
import random

def load_intents(file_path):
    with open(file_path, 'r') as file:
        intents = json.load(file)
    return intents

def process_user_input(user_input, nlp):
    doc = nlp(user_input.lower())
    return doc

def extract_keywords(doc, nlp):
    keywords = [token.text for token in doc if token.pos_ in ('NOUN', 'VERB', 'ADJ')]
    return keywords

def get_intent(intents, keywords):
    for intent in intents['intents']:
        for pattern in intent['patterns']:
            if any(keyword in pattern.lower() for keyword in keywords):
                return intent
    return None

def get_response(intent):
    if intent:
        responses = intent['responses']
        document_url = intent.get('document_url', None)
        video_url = intent.get('video_url', None)
        location = intent.get('location', None)
        image_url = intent.get('image_url', None)
        return random.choice(responses), document_url, video_url, location, image_url
    return "I'm sorry, I don't understand that.", None, None, None, None

def main():
    nlp = spacy.load('en_core_web_sm')
    intents = load_intents('/content/queries (2).json')

    print("Chatbot: Hi! How can I help you? (Type 'exit' to end)")

    while True:
        user_input = input("You: ").lower()

        if user_input == 'exit':
            print("Chatbot: Goodbye!")
            break

        doc = process_user_input(user_input, nlp)
        keywords = extract_keywords(doc, nlp)
        intent = get_intent(intents, keywords)
        response, document_url, video_url, location, image_url = get_response(intent)

        print("Chatbot:", response)

        # Check if "location" exists and print it
        if location:
            print("Location:", location)

        # Check if "document_url" exists and print it
        if document_url:
            print("Document URL:", document_url)

        # Check if "video_url" exists and print it
        if video_url:
            print("Video URL:", video_url)

        # Check if "image_url" exists and print it
        if image_url:
            print("Image URL:", image_url)

if __name__ == "__main__":
    main()






/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


Chatbot: Hi! How can I help you? (Type 'exit' to end)
You: location
Chatbot: I'm sorry, I don't understand that.
You: staff
Chatbot: We have highly qualified and well experienced staff members to teach and guide the students in their academic and professional career.Out of 513 staff members 302 staff members are Ph.D holders and 161 staff members are currently pursuing their doctoral programme
You: abhs
Chatbot: Yes, The Advisory Bureau for Higher Studies (ABHS) is a separate cell which takes care of all the requirements of the students for pursuing higher studies in India and around the Globe. Campus coaching classes for GATE, GRE, TOEFL & CAT are conducted for the benefit of our hostel students.
You: package
Chatbot: Adobe Systems has offered the  highest CTC of 40 lakhs per annum.
You: executive
Chatbot: Mrs.S.Jessie Priya M.com is the Executive Director of the institution.
Image URL: ['https://www.stjosephstechnology.ac.in/web/img/team/21.jpg']
You: name
Chatbot: The name of the co